## Outline
0. Pattern matching basics
1. Lists and Pattern Matching
2. Higher-order Functions
3. Abstract Syntax Tree


## Pattern matching basics

In [4]:
// What should be the output?
val x = 10 // What if x is 2
x match {
    case 1 => println(s"$x == 1")
    case 2 => println(s"$x == 2")
    case 3 => println(s"$x == 3")
    case _ => println(s"$x")
}

10


x: Int = 10

In [6]:
// What should be the output?
val x = 10 // What if x is 2
x match {
    case _ if x > 5 => println(s"$x > 5") 
    case _ if x > 1 => println(s"$x > 1")
    case _ => println(s"$x")
}

10 > 5


x: Int = 10

In [7]:
// What should be the output?
val x = 10
x match {
    case y if x > 5 => println(s"$y > 5") 
    case _ if x > 1 => println(s"$x > 1")
    case _ => println(s"$x")
}

10 > 5


x: Int = 10

In [8]:
// What should be the output?
val x = 10
x match {
    case y@(10 | 20 | 30) => println(s"$y is either 10 or 20 or 30") 
    case _ => println(s"$x")
}

10 is either 10 or 20 or 30


x: Int = 10

In [9]:
// What should be the output?
val x = 10
x match {
    case y@(1 | 2 | 3) => println(s"$y is either 1 or 2 or 3") 
    case _ => println(s"$x")
}

10


x: Int = 10

In [10]:
// What should be the output?
val x = 10
x match {
    case y@(1 | 2 | _) => println(s"$y is either 1 or 2 or whatever") 
    case _ => println(s"$x")
}

10 is either 1 or 2 or whatever


x: Int = 10

## Lists and Pattern Matching

Our inductive structure of choice for this problem is a linked list. The below code defines the structure.

In [11]:
sealed trait NumList
case object MyNil extends NumList
case class MyCons(firstElement : Int, restOfList : NumList) extends NumList

defined trait NumList
defined object MyNil
defined class MyCons

Since it's an inductive structure, we can write a grammar for it:

$$
\begin{array}{rcl}
    \textbf{NumList} & \rightarrow & MyNil \\
                     &           | & MyCons(\textbf{Int}, \textbf{NumList}) \\
\end{array}
$$

### Exercise: Writing lists
Write out the following lists with our `NumList` class.

1. `[]` (The empty list)
2. `[1, 2, 3]`

In [12]:
val ans_1 = MyNil
val ans_2 = MyCons(1,MyCons(2,MyCons(3,MyNil)))

ans_1: MyNil.type = MyNil
ans_2: MyCons = MyCons(1, MyCons(2, MyCons(3, MyNil)))

### Exercise: Length of list
Implement a `myLength` function for our list type using [pattern matching](https://docs.scala-lang.org/tour/pattern-matching.html).

In [13]:
// Is this function recursive?
// Is it tail recursive?

def myLength(list : NumList) : Int = {
    list match {
        case MyNil => 0
        case MyCons(firstElement, restOfList) => 1 + myLength(restOfList)
    }
}

defined function myLength

In [ ]:
assert(myLength(MyNil) == 0)
assert(myLength(MyCons(1, MyCons(2, MyNil))) == 2)

### Exercise: Translate to built in lists
Rewrite `length` to use [Scala's list class](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html), which is very similar to the one defined above. The table below shows the equivalences:

|`NumList`      | `List[Int]`|
|---------------|------------|
|`MyNil`        | `Nil`      |
|`MyCons(a, b)` | `a :: b`   |

In [18]:
def length(list : List[Int]) : Int = {
    list match {
        case Nil => 0
        case firstElement :: restOfList => 1 + length(restOfList)
    }
}

defined function length

In [19]:
assert(length(List()) == 0)
assert(length(List(1, 2, 3)) == 3)

## Abstract Syntax Trees (AST)
We'll be reusing the AST for sets from the previous recitation (without the complement). For more on ASTs, see
https://en.wikipedia.org/wiki/Abstract_syntax_tree

$$
\begin{array}{rcl}
    \textbf{Set} & \rightarrow & Empty \\
                 &           | & Cons(\textbf{Set}, \textbf{Int}) \\
                 &           | & Intersection(\textbf{Set}, \textbf{Set}) \\
                 &           | & Union(\textbf{Set}, \textbf{Set}) \\
                 &           | & Subtraction(\textbf{Set}, \textbf{Set}) \\
\end{array}
$$

In [20]:
sealed trait MySet
case object Empty extends MySet
case class Cons(s: MySet, n: Int) extends MySet
case class Intersection(s1: MySet, s2: MySet) extends MySet
case class Union(s1: MySet, s2: MySet) extends MySet
case class Subtraction(s1: MySet, s2: MySet) extends MySet

defined trait MySet
defined object Empty
defined class Cons
defined class Intersection
defined class Union
defined class Subtraction

### Exercise: Interpreter

Scala Set API -- https://docs.scala-lang.org/overviews/collections/sets.html

Write an interpreter for the language of sets you've created. Some useful methods of Sets:

```
Set.+:           (Set[Int], Int)            => Set[Int]
Set.intersect:   (Set[Int], Set[Int])       => Set[Int]
Set.union:       (Set[Int], Set[Int])       => Set[Int]
Set.diff:        (Set[Int], Set[Int])       => Set[Int]
```

In [24]:
import scala.collection.immutable.HashSet
def eval(set_expression: MySet): Set[Int] = {
    set_expression match {
        case Empty => Set[Int]()
        case Cons(s, n) => eval(s) + n
        case Intersection(s1, s2) => eval(s1) & eval(s2)
        case Union(s1, s2) => eval(s1) union eval(s2)
        case Subtraction(s1, s2) => eval(s1) diff eval(s2)
    }
}

import scala.collection.immutable.HashSet

defined function eval

In [23]:
val set_1_2 = Cons(Cons(Empty, 1), 2)
val set_1_2_3 = Cons(Cons(Cons(Empty, 1), 2), 3)
val set_3 = Cons(Empty, 3)

assert(eval(Empty) == Set())
assert(eval(set_1_2) == Set(1, 2))
assert(eval(Union(set_1_2, set_3)) == Set(1, 2, 3))
assert(eval(Intersection(set_1_2, set_1_2_3)) == Set(1, 2))
assert(eval(Subtraction(set_1_2, set_1_2_3)) == Set())

set_1_2: Cons = Cons(Cons(Empty, 1), 2)
set_1_2_3: Cons = Cons(Cons(Cons(Empty, 1), 2), 3)
set_3: Cons = Cons(Empty, 3)